# UNet4.0

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Paths
dataset_root = "/content/sentinel_sar_dataset/v_2/urban"
image_size = (256, 256)
num_images = 4000  # Training with 4000 images

# Load images
def load_images(folder, num_images):
    image_files = sorted(os.listdir(folder))[:num_images]
    images = [img_to_array(load_img(os.path.join(folder, img), target_size=image_size)) for img in image_files]
    return np.array(images) / 255.0  # Normalize

# Load SAR (input) and Optical (target)
X = load_images(f"{dataset_root}/s1", num_images)
Y = load_images(f"{dataset_root}/s2", num_images)

print(f"✅ Loaded {X.shape[0]} SAR images & {Y.shape[0]} Optical images.")


# 🧩 Cell 1: Install and Import Dependencies

In [ ]:
!pip install segmentation-models

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split
import numpy as np


# 📦 Cell 2: Prepare Dataset (Split into Train/Val/Test)

In [ ]:
# Assuming X and Y are your full datasets with 4000 images
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)  # 80% train
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)  # 10% val, 10% test


# 🌀 Cell 3: Data Augmentation for Training Set

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

# Combine images and masks for flow()
train_generator = train_datagen.flow(X_train, Y_train, batch_size=16)
val_generator = val_datagen.flow(X_val, Y_val, batch_size=16)


# 🧠 Cell 4: Define Loss Functions

In [ ]:
vgg = VGG16(include_top=False, weights="imagenet", input_shape=(256, 256, 3))
vgg.trainable = False
perceptual_model = Model(inputs=vgg.input, outputs=vgg.get_layer("block3_conv3").output)

def ssim_loss(y_true, y_pred):
    return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)

def perceptual_loss(y_true, y_pred):
    return K.mean(K.abs(perceptual_model(y_true) - perceptual_model(y_pred)))

def combined_loss(y_true, y_pred):
    return 0.5 * ssim_loss(y_true, y_pred) + 0.5 * perceptual_loss(y_true, y_pred)


# 🏗️ Cell 5: Define U-Net Model (With Dropout + L2)

In [ ]:
def unet_model(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(0.3)(p1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(0.3)(p2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(0.3)(p3)

    # Bottleneck
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(c4)

    # Decoder
    u5 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c3])
    u5 = Dropout(0.3)(u5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(u5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c2])
    u6 = Dropout(0.3)(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c1])
    u7 = Dropout(0.3)(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(c7)

    outputs = Conv2D(3, (1, 1), activation='sigmoid')(c7)

    return Model(inputs, outputs)


# 🧪 Cell 6: Compile & Train with EarlyStopping + Checkpoint

In [ ]:
model = unet_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss=combined_loss, metrics=['mae'])

early_stop = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_unet_model.h5", save_best_only=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stop, checkpoint]
)


# 📊 Cell 7: Final Evaluation on Test Set

In [ ]:
test_loss = model.evaluate(X_test, Y_test)
print("Test Loss:", test_loss)


# 🌟 Cell 8: BONUS – Use Pretrained U-Net (ResNet34 encoder)

In [1]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing

preprocess_input = get_preprocessing('resnet34')

X_train_prep = preprocess_input(X_train)
X_val_prep = preprocess_input(X_val)
X_test_prep = preprocess_input(X_test)

model_resnet = Unet(
    backbone_name='resnet34',
    encoder_weights='imagenet',
    classes=3,
    activation='sigmoid'
)

model_resnet.compile(optimizer=Adam(learning_rate=0.001), loss=combined_loss, metrics=['mae'])

history_resnet = model_resnet.fit(
    X_train_prep, Y_train,
    validation_data=(X_val_prep, Y_val),
    epochs=50,
    batch_size=16,
    callbacks=[early_stop, checkpoint]
)
test_loss = model_resnet.evaluate(X_test, Y_test)
print("Test Loss:", test_loss)

ModuleNotFoundError: No module named 'segmentation_models'